In [167]:
import requests, sys
import pandas as pd
import json

In [168]:
var = pd.read_csv("../variants.tsv", sep="\t")

In [169]:
hgnc_gene_chr_map = pd.read_csv("hgnc_gene_chr_map.tsv", sep="\t")

# Cleanup hgnc gene map entries and remove any entries without a chromosome location

In [170]:
hgnc_gene_chr_map.drop(hgnc_gene_chr_map[hgnc_gene_chr_map.Chromosome.isna()].index, inplace=True)

In [172]:
hgnc_gene_chr_map['Chromosome'] = hgnc_gene_chr_map['Chromosome'].str.replace("p.*|q.*","")

In [173]:
hgnc_gene_chr_map.rename(columns={'Approved symbol':'Gene'}, inplace=True)

# Combine variants and hgnc map so that we know which chromosome a gene belongs to

In [174]:
var = var.merge(hgnc_gene_chr_map, how='left', on='Gene')

In [176]:
var['hgvs_g'] = var.apply(lambda x: "chr{}:{}".format(x['Chromosome'], x['DNA_change_genomic_hg19']), axis=1)

In [177]:
hgvs_list = var['hgvs_g'][~var['hgvs_g'].str.contains("?", regex=False)]

In [178]:
hgvs_list.drop_duplicates().to_csv("../hgvslist.txt", index=False, header=False)

In [182]:
var.drop_duplicates().to_csv("variants2.tsv", index=False, header=True, sep="\t")

# Obtain VEP predictions

In [250]:
import re

In [251]:
vep = pd.read_csv("variants2.txt", sep="\t")

In [252]:
vcf = pd.read_csv('variants2.vcf', skiprows=3, sep="\t")

In [253]:
vcf = vcf[['ID','#CHROM','POS','REF','ALT']].rename(columns={'ID':'hgvs_g','#CHROM':'CHROM'}).drop_duplicates()

In [254]:
var['key'] = var.apply(lambda x: "{}-{}".format(re.sub('\..+',"",x['Transcript']), x['hgvs_g']), axis=1)

In [255]:
vep['key'] = vep.apply(lambda x: "{}-{}".format(re.sub('\..+',"",x['Feature']), x['#Uploaded_variation']), axis=1)

In [256]:
vep.drop(columns=['Gene'], inplace=True)

In [257]:
var2 = var.merge(vep, how='left', on='key')

In [258]:
var3 = var2.merge(vcf, how='left', on='hgvs_g')

In [259]:
var3.drop(columns=['#Uploaded_variation','key'], inplace=True)

In [260]:
var3.to_csv("variants_vep.tsv", index=False, header=True, sep="\t")

In [261]:
var3['Clinical_classification'].value_counts()

VUS                                                                                          4363
-                                                                                            1790
likely benign                                                                                1778
benign                                                                                       1472
pathogenic                                                                                   1078
                                                                                             ... 
likely benign, likely pathogenic, benign, likely pathogenic (recessive)                         1
pathogenic (recessive), VUS, likely pathogenic (recessive), likely pathogenic, pathogenic       1
pathogenic (recessive), likely pathogenic, pathogenic                                           1
pathogenic, likely pathogenic (recessive), pathogenic (recessive)                               1
VUS, pathogenic, lik